### Contents

* Inheritance
    * Polymorphism
    * Duck typing
    * Multiple inheritance (MRO + mixins)
* SOLID
    * Single Responsibility Principle
    * Open/Closed Principle
    * Liskov Substitution Principle
    * Interface Segregation Principle
    * Dependency Inversion Principle

## Inheritance (~20:05 (30min))

In [ ]:
# What's wrong?
class CircleBad:
    def __init__(self, x, y, r):
        self.x, self.y, self.r = x, y, r
    def move(self, dx, dy):
        self.x += dx; self.y += dy
    def draw(self):
        return f"CircleBad({self.x}, {self.y}, r={self.r})"

class RectangleBad:
    def __init__(self, x, y, w, h):
        self.x, self.y, self.w, self.h = x, y, w, h
    def move(self, dx, dy):
        self.x += dx; self.y += dy
    def draw(self):
        return f"RectangleBad({self.x}, {self.y}, {self.w}x{self.h})"

c = CircleBad(0, 0, 3)
r = RectangleBad(10, 5, 4, 2)
c.move(1, 1); r.move(-2, 3)
print(c.draw())
print(r.draw())


Both classes implement `move` method. Code is duplicated.

**Inheritance** is a mechanism in programming languages that allows a new class to reuse and extend the definitions of an existing class.

In [ ]:
class Shape:
    def __init__(self, x: float, y: float):
        self.x, self.y = x, y
    def move(self, dx: float, dy: float):
        self.x += dx; self.y += dy


class Circle(Shape):
    pass

c = Circle(1, 2)
print(c.x, c.y)
c.move(1, 1)
print(c.x, c.y)

In [ ]:
# full example

class Shape:
    def __init__(self, x: float, y: float):
        self.x, self.y = x, y
    def move(self, dx: float, dy: float):
        self.x += dx; self.y += dy

class Circle(Shape):
    def __init__(self, x: float, y: float, r: float):
        super().__init__(x, y)            # NOTE: base init
        self.r = r
    def draw(self) -> str:
        return f"Circle({self.x}, {self.y}, r={self.r})"

class Rectangle(Shape):
    def __init__(self, x: float, y: float, w: float, h: float):
        super().__init__(x, y)
        self.w, self.h = w, h
    def draw(self) -> str:
        return f"Rectangle({self.x}, {self.y}, {self.w}x{self.h})"

c = Circle(1, 1, 3); r = Rectangle(8, 8, 4, 2)
c.move(1, 0.5); r.move(-1, -2)
print(c.draw())
print(r.draw())


Overriding methods

In [8]:
class A:
    def foo(self):
        print("A.foo")

class B(A):
    def foo(self):
        print("B.foo")

class C(A):
    pass

a = A()
b = B()
c = C()

a.foo()
b.foo()
c.foo()

A.foo
B.foo
A.foo


In [11]:
class PrettyCircle(Circle):
    def draw(self) -> str:
        # different string format — same method name
        return f"◯ at ({self.x}, {self.y}) radius={self.r}"

pc = PrettyCircle(0, 0, 2)
print(pc.draw())


◯ at (0, 0) radius=2


`super()` gives you the parent class

In [17]:
class Shape:
    def __init__(self, x: float, y: float):
        self.x, self.y = x, y
    def move(self, dx: float, dy: float):
        self.x += dx; self.y += dy

class Circle(Shape):
    def __init__(self, x, y, r):
        # forgot to initialize the base!
        self.r = r

bad = Circle(0, 0, 5)
try:
    bad.move(1, 1)  # this will fail because x/y are missing
except Exception as e:
    print("❌ Missing base init →", type(e).__name__, e)


❌ Missing base init → AttributeError 'Circle' object has no attribute 'x'


Here, we override the `__init__` method. We don't call parent's `__init__` method by default.

In [18]:

class CircleWithSuper(Shape):
    def __init__(self, x, y, r):
        super().__init__(x, y)  # super() returns the parent class (Shape)
        self.r = r

good = CircleWithSuper(0, 0, 5)
good.move(1, 1)
print("✅ After move:", good.x, good.y)


✅ After move: 1 1


`super()` returns a proxy object that accesses the parent class.

Note: In MRO, we'll see that it's a little bit more complicated.

In [20]:
class A(Shape):
    def __init__(self, x, y):
        print("super", super())
        print("super().__init__", super().__init__)
        print("super class:", super().__class__.__name__)

a = A(1, 2)

super <super: <class 'A'>, <A object>>
super().__init__ <bound method Shape.__init__ of <__main__.A object at 0x1080818e0>>
super class: super


`issubclass(type_a, type_b)` tells you whether `type_a` is a subclass of `type_b`.

In [22]:
class A:
    pass

class B(A):
    pass

class C(B):
    pass

print(f"{issubclass(C, A)=}")
print(f"{issubclass(C, B)=}")
print(f"{issubclass(C, C)=}")
print(f"{issubclass(A, B)=}")

issubclass(C, A)=True
issubclass(C, B)=True
issubclass(C, C)=True
issubclass(A, B)=False


### Polymorphism

**Polymorphism** means writing code that works with different types of objects through a common interface, letting each object decide its own specific behavior.

In [23]:
class Base:
    def __init__(self, x, y):
        self.x, self.y = x, y

    def draw(self):
        return f"Base({self.x}, {self.y})"

class Child(Base):
    def draw(self):
        return f"Child({self.x}, {self.y})"

class SecondChild(Child):
    def draw(self):
        return f"SecondChild({self.x}, {self.y})"


def render_all(shapes):
    return [s.draw() for s in shapes]

items = [Base(0,0), Child(1,1), SecondChild(2,2)]
for line in render_all(items):
    print(line)


Base(0, 0)
Child(1, 1)
SecondChild(2, 2)


### Abstraction

**Abstraction** lets us describe what an object does, without saying how it does it. An abstract class defines this contract, while concrete classes provide the actual implementation.

![abstraction](./abstraction.png)

In [25]:
from abc import ABC, abstractmethod

class Animal(ABC):
    @abstractmethod
    def make_sound(self):
        pass

class Dog(Animal):
    def make_sound(self):
        return "Woof"

class Cat(Animal):
    def make_sound(self):
        return "Meow"

dog = Dog()
cat = Cat()

print(dog.make_sound())
print(cat.make_sound())

Woof
Meow


In [27]:
animal = Animal()  # can't instantiate abstract class

TypeError: Can't instantiate abstract class Animal without an implementation for abstract method 'make_sound'

In [28]:
# obligated to implement all abstract methods: our contract

class Bird(Animal):  # bird class is abstract since we don't implement make_sound
    pass

bird = Bird()

TypeError: Can't instantiate abstract class Bird without an implementation for abstract method 'make_sound'

optional

abstract property

In [29]:
from abc import ABC, abstractmethod

class Animal(ABC):
    @abstractmethod
    def print_sound(self):
        pass

class Dog(Animal):
    def print_sound(self):
        print("Dog says Woof")

class Cat(Animal):
    def print_sound(self):
        print("Cat says Meow")

dog = Dog()
cat = Cat()

dog.print_sound()
cat.print_sound()

Dog says Woof
Cat says Meow


In [30]:
from abc import ABC, abstractmethod

class Animal(ABC):
    def print_sound(self):
        class_name = self.__class__.__name__
        print(f"{class_name} says {self.sound}")
    
    @property  # order matters: abstract method must be before property
    @abstractmethod
    def sound(self):
        pass

class Dog(Animal):
    @property
    def sound(self):
        return "Woof"

class Cat(Animal):
    @property
    def sound(self):
        return "Meow"

dog = Dog()
cat = Cat()

dog.print_sound()
cat.print_sound()

Dog says Woof
Cat says Meow


### Duck typing

> If it walks like a duck, quacks like a duck, and looks like a duck, then it is a duck.

Idea: you can use polymorphism on objects that have the same interface, even if they are not related by inheritance

In [31]:
# Q: do you see a problem here?
from io import StringIO
import sys

def log(writable, msg: str) -> None:
    writable.write(msg + "\n")  # duck-typed: any object with .write works

# Example 1: StringIO buffer (in-memory text stream)
buf = StringIO()
log(buf, "hello")
log(buf, "world")
print("Buffer contents:")
print(buf.getvalue())

# Example 2: Writing to a file
with open("log.txt", "w") as f:
    log(f, "First line in file")
    log(f, "Second line in file")
!cat log.txt

# Example 3: Writing to standard output
log(sys.stdout, "\nThis goes directly to the console!")
log(sys.stdout, "Duck typing is present here too.")


Buffer contents:
hello
world

First line in file
Second line in file

This goes directly to the console!
Duck typing is present here too.


In [32]:
# the types above are unrelated

buffer_type = type(StringIO())
file_type = type(open("log.txt"))
stdout_type = type(sys.stdout)

print(f"{buffer_type=}")
print(f"{file_type=}")
print(f"{stdout_type=}")

print(f"{issubclass(buffer_type, file_type)=}")
print(f"{issubclass(file_type, buffer_type)=}")
print(f"{issubclass(buffer_type, stdout_type)=}")
print(f"{issubclass(stdout_type, buffer_type)=}")



buffer_type=<class '_io.StringIO'>
file_type=<class '_io.TextIOWrapper'>
stdout_type=<class 'ipykernel.iostream.OutStream'>
issubclass(buffer_type, file_type)=False
issubclass(file_type, buffer_type)=False
issubclass(buffer_type, stdout_type)=False
issubclass(stdout_type, buffer_type)=False


Problem: implicity

When we use duck typing, how are we guaranteed that the object has the required methods? \
Usually, we use type annotations for this. \
What type annotation should we use?

Solution: `typing.Protocol` — structural contract

In [33]:
%%writefile log.py
from typing import Protocol, Any
from io import StringIO

class Writable(Protocol):
    def write(self, s: str, /) -> Any:  # note: s should be positional-only
        ...

def log_typed(writable: Writable, msg: str) -> None:
    writable.write(msg + "\n")

buf = StringIO()
log_typed(buf, "hello")
log_typed(buf, "world")
print("Buffer contents:")
print(buf.getvalue())


with open("log.txt", "w") as f:
    log_typed(f, "hello")
    log_typed(f, "world")

print("File contents:")
with open("log.txt", "r") as f:
    print(f.read())

Writing log.py


In [34]:
!python log.py

Buffer contents:
hello
world

File contents:
hello
world



In [35]:
!pyrefly check log.py

 INFO 0 errors░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░      56/60     


### Multiple Inheritance

In [41]:
class Vehicle:
    def refuel(self):
        print("Refueling somehow")

class Car(Vehicle):
    def drive(self):
        print("Driving on the road")

    def refuel(self):
        print("Refueling with gasoline")


class Plane(Vehicle):
    def fly(self):
        print("Flying in the sky")

    def refuel(self):
        print("Refueling with aviation fuel")


class FlyingCar(Car, Plane):
    pass  # no need to implement anything


fc = FlyingCar()
fc.drive()   # From Car
fc.fly()     # From Plane
fc.refuel()  # What's here?

Driving on the road
Flying in the sky
Refueling with gasoline


In [42]:
fc.refuel()  # What's here?

Refueling with gasoline


MRO (Method Resolution Order)

In [44]:
class D:
    def hello(self): print("D")

class B(D):
    pass

class C(D):
    pass

class A(B, C):
    pass

print(*A.mro(), sep="\n")

<class '__main__.A'>
<class '__main__.B'>
<class '__main__.C'>
<class '__main__.D'>
<class 'object'>


### Mixin

**Mixin** is a class that is used to add functionality (usually one method) to another class.

In [45]:
class Shape:
    def __init__(self, x, y):
        self.x, self.y = x, y

    def move(self, dx, dy):
        self.x += dx
        self.y += dy

In [47]:
# BAD: duplication and SRP violation (geometry + IO mixed)
class CircleWithSerializeBad(Shape):
    def __init__(self, x, y, r):
        super().__init__(x, y); self.r = r
    def serialize(self):
        return {"type": "circle", "x": self.x, "y": self.y, "r": self.r}

class RectangleWithSerializeBad(Shape):
    def __init__(self, x, y, w, h):
        super().__init__(x, y); self.w, self.h = w, h
    def serialize(self):
        return {"type": "rectangle", "x": self.x, "y": self.y, "w": self.w, "h": self.h}

print(CircleWithSerializeBad(0,0,1).serialize())
print(RectangleWithSerializeBad(1,1,2,3).serialize())


{'type': 'circle', 'x': 0, 'y': 0, 'r': 1}
{'type': 'rectangle', 'x': 1, 'y': 1, 'w': 2, 'h': 3}


Solution: mixin

In [48]:
class SerializableMixin:
    def serialize(self) -> dict:
        # simple example: export public attributes
        return {k: v for k, v in self.__dict__.items() if not k.startswith("_")}


class CircleMI(SerializableMixin, Shape):
    def __init__(self, x: float, y: float, r: float, **kwargs):
        super().__init__(x=x, y=y, **kwargs)  # note: **kwargs to show that sometimes we don't know what arguments should be passed to the parent class
        self.r = r

class RectangleMI(SerializableMixin, Shape):
    def __init__(self, x: float, y: float, w: float, h: float, **kwargs):
        super().__init__(x=x, y=y, **kwargs)
        self.w, self.h = w, h

print(CircleMI(0,0,2).serialize())
print(RectangleMI(1,1,2,3).serialize())


{'x': 0, 'y': 0, 'r': 2}
{'x': 1, 'y': 1, 'w': 2, 'h': 3}


## SOLID (~20:25 (20min))

### S — Single Responsibility Principle

what's wrong?

In [50]:
from dataclasses import dataclass

@dataclass
class Order:
    items: list[str]
    total: float
    email: str

class OrderProcessor:
    def process(self, order):
        if not order.items:
            raise ValueError("Empty order")         # validation
        print(f"Charging {order.total}...")         # payment
        print(f"Emailing receipt to {order.email}") # notification

# what if I want to change the validation logic? Or the payment logic?

op = OrderProcessor()
order = Order(items=['apple', 'banana'], total=100, email="test@test.com")
op.process(order)

Charging 100...
Emailing receipt to test@test.com


SRP - A class should serve one specific purpose

In [53]:
class OrderValidator:
    def validate(self, order):
        if not order.items:
            raise ValueError("Empty order")

class PaymentGateway:
    def charge(self, amount):
        print(f"Charging {amount}...")

class AlternativePaymentGateway:
    def charge(self, amount):
        print(f"Super-charging {amount}...")

class Notifier:
    def send_receipt(self, email):
        print(f"Email sent to {email}")

class OrderProcessor:
    def __init__(self, validator, gateway, notifier):
        self.validator = validator
        self.gateway = gateway
        self.notifier = notifier

    def process(self, order):
        self.validator.validate(order)
        self.gateway.charge(order.total)
        self.notifier.send_receipt(order.email)

op = OrderProcessor(OrderValidator(), AlternativePaymentGateway(), Notifier())
op.process(order)

Super-charging 100...
Email sent to test@test.com


In [54]:
class AnotherPaymentGateway:
    TAX = 0.1
    def charge(self, amount):
        amount_with_tax = amount * (1 + self.TAX)
        print(f"Charging {amount_with_tax:.2f} (with tax {self.TAX*100}%)...")


op = OrderProcessor(OrderValidator(), AnotherPaymentGateway(), Notifier())
op.process(order)

Charging 110.00 (with tax 10.0%)...
Email sent to test@test.com


### O — Open/Closed Principle

Every new discount requires editing this function (fragile if/elif chain).

In [56]:
def discount(amount, kind):
    if kind == "vip":
        return amount * 0.8
    elif kind == "black_friday":
        return amount * 0.5
    elif kind == "employee":
        return amount * 0.7
    # ...keep adding branches forever
    return amount

discount(100, "vip")

80.0

Software entities should be open for extension, closed for modification. \
In simple words: add new things without touching old, working code

Extension: new functionality (new classes) \
Modification: existing functionality (existing classes)










In [58]:
from abc import ABC, abstractmethod

class DiscountPolicy(ABC):
    @abstractmethod
    def apply(self, amount: float) -> float: ...

class NoDiscount(DiscountPolicy):
    def apply(self, amount): return amount

class VipDiscount(DiscountPolicy):
    def apply(self, amount): return amount * 0.8

class BlackFriday(DiscountPolicy):
    def apply(self, amount): return amount * 0.5

def price(amount: float, policy: DiscountPolicy) -> float:
    return policy.apply(amount)

policy = VipDiscount()
price(100, policy)

80.0

### L — Liskov Substitution Principle

In [61]:
class Rectangle:
    def __init__(self, w=0, h=0): self.w, self.h = w, h
    def set_width(self, w):  self.w = w
    def set_height(self, h): self.h = h

class Square(Rectangle):
    def set_width(self, w):  self.w = self.h = w
    def set_height(self, h): self.w = self.h = h

def layout_button(rect: Rectangle):
    # UI code expects an exact size: 100x30
    rect.set_width(100)
    rect.set_height(30)
    assert (rect.w, rect.h) == (100, 30), "Layout broken!"

btn = Rectangle()
layout_button(btn)         # OK

sq_btn = Square()
layout_button(sq_btn)      # AssertionError: becomes 30x30 or 100x100


AssertionError: Layout broken!

If code works with the parent, it should work as well with any child.

If `Square` is a child of `Rectangle`, then we expect that whenever we use `Rectangle`, we can use `Square` as well. \
However, it's not true. That's why Liskov Substitution Principle is violated.

### I — Interface Segregation Principle

In [62]:
class Worker:
    def work(self): ...
    def eat(self): ...
    def sleep(self): ...

# problem: client is forced to implement unwanted methods
class Robot(Worker):
    def work(self): print("Assembling")
    def eat(self): raise NotImplementedError  # forced
    def sleep(self): raise NotImplementedError


Principle: don’t make me implement what I don’t use



In [63]:
from typing import Protocol

class Workable(Protocol):
    def work(self) -> None: ...

class Eatable(Protocol):
    def eat(self) -> None: ...

class Sleepable(Protocol):
    def sleep(self) -> None: ...

class Human(Workable, Eatable, Sleepable):
    def work(self): print("Coding")
    def eat(self): print("Lunch")
    def sleep(self): print("Zzz")

class Robot(Workable):
    def work(self): print("Assembling")


### D — Dependency Inversion Principle

In [64]:
class MySQLReportRepo:
    def save(self, report): print("Saved to MySQL")

class ReportService:
    def __init__(self):
        self.repo = MySQLReportRepo()  # hard dependency; what if we want to save to another database?

    def generate(self, data):
        report = f"report:{data}"
        self.repo.save(report)


Depend on abstractions, not concretions. \
High-level rules shouldn’t know low-level details. Both sides depend on a small interface so you can swap implementations freely.

In [65]:
from typing import Protocol

class ReportRepo(Protocol):
    def save(self, report: str) -> None: ...

class MySQLReportRepo:
    def save(self, report): print("Saved to MySQL")

class InMemoryReportRepo:
    def __init__(self): self.store = []
    def save(self, report): self.store.append(report)

class ReportService:
    def __init__(self, repo: ReportRepo):
        self.repo = repo  # inversion via constructor

    def generate(self, data: dict):
        report = f"report:{data}"
        self.repo.save(report)

# Usage:
# ReportService(MySQLReportRepo())  # prod
# ReportService(InMemoryReportRepo())  # tests


*Note:* You get to know the idea better as you work on more and more projects.